In [1]:
import gym
import wandb
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define Policy and REINFORCE classes

class Policy(nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Policy, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

class REINFORCE:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma, device):
        self.policy_net = Policy(state_dim, hidden_dim, action_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.device = device

    def take_action(self, state):
        state = torch.tensor(state.reshape(1, -1), dtype=torch.float).to(self.device) 
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs) 
        action = action_dist.sample()
        return action.item()  

    def update(self, transition_dict):
        reward_list = transition_dict['rewards']
        state_list = transition_dict['states']
        action_list = transition_dict['actions']

        G = 0
        self.optimizer.zero_grad()
        for i in reversed(range(len(reward_list))):
            reward = reward_list[i]
            state = torch.tensor(state_list[i].reshape(1, -1), dtype=torch.float).to(self.device)
            action = torch.tensor(action_list[i]).view(-1, 1).to(self.device)
            log_prob = torch.log(self.policy_net(state).gather(1, action))
            G = self.gamma * G + reward
            loss = -log_prob * G
            loss.backward()
        self.optimizer.step()

# Hyperparameters
learning_rate = 1e-3
gamma = 0.99
hidden_dim = 128
num_pbar = 10
num_episodes = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env_name = "CartPole-v1"

# Define the sweep configuration
sweep_config = {
    "method": "bayes",
    "metric": {"name": "regret", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 1e-5, "max": 1e-2},
        "hidden_dim": {"values": [64, 128, 256]}
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "max_iter": 100
}
}
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config)

def train(learning_rate, hidden_dim):
    env = gym.make(env_name)
    env.reset(seed=0)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = REINFORCE(state_dim, hidden_dim, action_dim, learning_rate, gamma, device)
    return_list = []

    for i in range(num_pbar):
        with tqdm(total=int(num_episodes/num_pbar), desc='Iteration %d' % i) as pbar:
            for i_episode in range(int(num_episodes/num_pbar)):
                episode_return = 0
                transition_dict = {
                    "states": [],
                    "actions": [],
                    "next_states": [],
                    "rewards": [],
                    "dones": []
                }
                state, _ = env.reset()
                terminated, truncated = False, False
                while not terminated and not truncated:
                    action = agent.take_action(state)
                    next_state, reward, terminated, truncated, _ = env.step(action)
                    transition_dict["states"].append(state)
                    transition_dict["actions"].append(action)
                    transition_dict["next_states"].append(next_state)
                    transition_dict["rewards"].append(reward)
                    transition_dict["dones"].append(terminated)
                    state = next_state
                    episode_return += reward
                return_list.append(episode_return)
                agent.update(transition_dict)
                if (i_episode+1)%10 == 0:
                    pbar.set_postfix({
                        'episode_return': '%d' % (num_episodes/num_pbar*i+i_episode+1),
                        'return': '%.3f' % np.mean(return_list[-10:])
                    })
                pbar.update(1)

    avg = []
    for i in range(num_episodes):
        if i < 100:
            avg_return = np.mean(return_list[:i+1])
        else:
            avg_return = np.mean(return_list[i-100:i+1])
        avg.append(avg_return)

    regret = 0
    for i in avg:
        if i < 195:
            regret += 195 - i

    return regret

def run_training():
    config_defaults = {
        "learning_rate": 1e-3,
        "hidden_dim": 128
    }
    config = wandb.init(config=config_defaults)
    learning_rate = config.config["learning_rate"]
    hidden_dim = config.config["hidden_dim"]
    regret = train(learning_rate, hidden_dim)
    wandb.log({"regret": regret})

# Run the sweep
wandb.agent(sweep_id, function=run_training)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ebygp3ca
Sweep URL: https://wandb.ai/rl_shobhith/uncategorized/sweeps/ebygp3ca


wandb: Agent Starting Run: knroq4ij with config:
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.0008014309956249786
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shobhith-v (rl_shobhith). Use `wandb login --relogin` to force relogin


Iteration 0:   0%|          | 0/50 [00:00<?, ?it/s]C:\Users\Shobhith Vadlamudi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Iteration 9: 100%|██████████| 50/50 [00:11<00:00,  4.53it/s, episode_return=500, return=321.700]


regret,▁
regret,30714.25507


...

wandb: Agent Starting Run: ikbb4g52 with config:
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.000531331974149722
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Iteration 9: 100%|██████████| 50/50 [00:01<00:00, 27.49it/s, episode_return=500, return=72.100]


regret,▁
regret,78640.69602


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: p5qr5hqd with config:
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.0008842844680140334
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Iteration 9: 100%|██████████| 50/50 [00:16<00:00,  3.01it/s, episode_return=500, return=460.600]
